In [ ]:
import json, urllib
import pandas as pd
import numpy as np

In [ ]:
# Building blocks of the API calls
BASE_ENDPOINT = 'http://archive.org/wayback/available?url='
TIMESTAMP_ARG = '&timestamp='

# Any CSV formatted as per our usual project standards can go in here.
def CSV_PROCESS(csvlocation):
    URL_LIST = df_URLs = pd.read_csv(csvlocation, low_memory = False, usecols=["NCESSCH", "URL"]) 
    URL_LIST = URL_LIST['URL'].tolist()
    return URL_LIST

# This function takes a list of URLs, checks if each one is on the Wayback Machine, and if it is, then a constructed URL will be outputted.
# If the optional timestamp parameter is entered, it will check if a snapshot from that date is available. 
# Timestamps must be formatted as YYYYMMDD strings, not integers, otherwise it'll throw a SyntaxError.
# INPUT: Shallow Python list of string URLs
# OUTPUT: Shallow Python list of 
def URL_PROCESS(urls, timestamp = ""):
    result = []
    for url in urls:
        if timestamp:
            endpoint = BASE_ENDPOINT + url + TIMESTAMP_ARG + timestamp
        else:
            endpoint = BASE_ENDPOINT + url
        
        response = json.load(urllib.request.urlopen(endpoint))
        found = len(response['archived_snapshots']) > 0
        
        if found:
            result.append(response['archived_snapshots']['closest']['url']) 
            
    return result